<a href="https://colab.research.google.com/github/isaacmaruyama/nightlights/blob/main/blackmarble_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Overview
This is a Python script I wrote that uses nightlights data taken from NASA's Black Marble program to examine trends in luminosity for specific Chinese administrative states.



In [1]:
# Import library dependencies
import osgeo.gdal as gdal, numpy as np, os, ee, geemap

# Earth Engine authentification
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Ta68WPM0VpsXxVS5NyhRL_1yAM70d1fMZ_K_S4ALuZg&tc=20Qxzpuu4eSyFsbkai3ZGNBcffDi60sJuMX66tSKvqI&cc=Fev7AfM1tiu57wFCq1KYJWqZzTUHDH6BK5lEX1BY08c

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXng6CPiVGaBCQK_US5n8n0zLrlDwRCAd2u5h-7rE09UOgjmekntUMM

Successfully saved authorization token.


In [65]:
# Set time variables for luminosity comparison
month = '09' # 2-digit format
comparison_years = [2022, 2023] # 2 years in [old, new] format

# Create colormap based on radiance values
radiance_values = [-0.1, -0.05, 0.05, 0.1]
radiance_colormap = ['#0000FF', '#4000BF', '#800080', '#BF0040', '#FF0000']

# Create empty lists for comparing radiance levels
old_list = []
new_list = []

# Load a feature collection with Chinese administrative states
china_states = ee.FeatureCollection('FAO/GAUL/2015/level1') \
    .filter(ee.Filter.eq('ADM0_NAME', 'China'))

# Define style parameters for initial visualization
style_params = {'color': 'white', 'width': 1, 'fillColor': '00000000'}

# Define counter variable for loop
counter = 0

# Generate radiance data for each comparison period
for year in comparison_years:
  # Define the time range for filtering the image collection
  start_date = f'{year}-{month}-01'
  end_date = f'{year}-{month}-20'

  # Load an image collection with VIIRS monthly nightlights data
  nightlights = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG') \
      .filter(ee.Filter.date(start_date, end_date)) \
      .select('avg_rad')

  # Define visualization parameters for nightlights
  nightlight_vis_params = {'min': 0.0, 'max': 60.0}

  # Create a mosaic of nightlights and clip it to include only China for both years
  mosaic_nightlights = nightlights.mosaic()
  clipped_nightlights = mosaic_nightlights.clip(china_states)

  # Store separate mosaics for both years
  if counter == 0:
    old_nightlights = clipped_nightlights
  elif counter == 1:
    new_nightlights = clipped_nightlights

  # Reduce regions to compute the mean radiance within each Chinese state
  average_radiance = clipped_nightlights.reduceRegions(
      collection=china_states,
      reducer=ee.Reducer.mean(),
      scale=1000  # Scale can be adjusted based on data resolution
  )

  # Create a new feature collection with administrative state name and average radiance as properties
  radiance_collection = average_radiance.map(lambda feature: feature.select(**{
    'propertySelectors': ['ADM1_NAME', 'mean'],
    'newProperties': ['State Name', 'Avg Radiance']
    }))

  # Extract a list of features from the collection
  radiance_list = radiance_collection.getInfo()['features']

  # Loop through each feature and add its properties to the comparison lists
  for feature in radiance_list:
      name = feature['properties']['State Name']
      avg_rad = feature['properties']['Avg Radiance']

      # Add properties to the lists as tuples
      if counter == 0:
        old_list.append((name, avg_rad))
      elif counter == 1:
        new_list.append((name, avg_rad))

  counter += 1

# Make a list containing the change in radiance for each administrative state
difference_list = [(name, (value2 - value1)/value1) for (name, value1), (_, value2) in zip(old_list, new_list)]
print(difference_list)

[('Anhui Sheng', -0.09349946487636573), ('Beijing Shi', -0.022815718812826704), ('Chongqing Shi', 0.03904718942641669), ('Fujian Sheng', 0.09806465724124307), ('Gansu Sheng', 0.10941124429069228), ('Guangdong Sheng', 0.04782411818116257), ('Guangxi Zhuangzu Zizhiqu', 0.09661702674312798), ('Guizhou Sheng', 0.17494570270158372), ('Hainan Sheng', 0.12563631374170423), ('Hebei Sheng', 0.03286210539258551), ('Henan Sheng', -0.04586821262075667), ('Hubei Sheng', -0.01882396546120645), ('Hunan Sheng', -0.02370029272764351), ('Jiangsu Sheng', -0.02927370452385976), ('Jiangxi Sheng', 0.11729226171020815), ('Liaoning Sheng', -0.014026846953567077), ('Nei Mongol Zizhiqu', 0.05092549979450901), ('Ningxia Huizu Zizhiqu', 0.2561426162239362), ('Qinghai Sheng', 0.15896785321813064), ('Shaanxi Sheng', 0.0268346061308152), ('Shandong Sheng', 0.020014878679431997), ('Shanghai Shi', -0.019721221281452126), ('Shanxi Sheng', 0.016449584511891477), ('Sichuan Sheng', 0.6643025693356956), ('Tianjin Shi', -0.

In [66]:
# Extract radiance values from difference list
radiance_list = [entry[1] for entry in difference_list]

# Function to assign color to state based on radiance
def get_color(number):
  if number <= radiance_list[0]:
    return radiance_colormap[0]
  elif number <= radiance_list[1]:
    return radiance_colormap[1]
  elif number <= radiance_list[2]:
    return radiance_colormap[2]
  elif number <= radiance_list[3]:
    return radiance_colormap[3]
  elif number > radiance_list[3]:
    return radiance_colormap[4]

# Create a final list with name, radiance, and color for allstates
colormap_list = [(number, get_color(number)) for number in radiance_list]
final_list = [(name, avg_rad, color) for (name, avg_rad), (_, color) in zip(difference_list, colormap_list)]

print(final_list)

[('Anhui Sheng', -0.09349946487636573, '#0000FF'), ('Beijing Shi', -0.022815718812826704, '#4000BF'), ('Chongqing Shi', 0.03904718942641669, '#800080'), ('Fujian Sheng', 0.09806465724124307, '#BF0040'), ('Gansu Sheng', 0.10941124429069228, '#FF0000'), ('Guangdong Sheng', 0.04782411818116257, '#BF0040'), ('Guangxi Zhuangzu Zizhiqu', 0.09661702674312798, '#BF0040'), ('Guizhou Sheng', 0.17494570270158372, '#FF0000'), ('Hainan Sheng', 0.12563631374170423, '#FF0000'), ('Hebei Sheng', 0.03286210539258551, '#800080'), ('Henan Sheng', -0.04586821262075667, '#4000BF'), ('Hubei Sheng', -0.01882396546120645, '#800080'), ('Hunan Sheng', -0.02370029272764351, '#4000BF'), ('Jiangsu Sheng', -0.02927370452385976, '#4000BF'), ('Jiangxi Sheng', 0.11729226171020815, '#FF0000'), ('Liaoning Sheng', -0.014026846953567077, '#800080'), ('Nei Mongol Zizhiqu', 0.05092549979450901, '#BF0040'), ('Ningxia Huizu Zizhiqu', 0.2561426162239362, '#FF0000'), ('Qinghai Sheng', 0.15896785321813064, '#FF0000'), ('Shaanxi S

In [69]:
# Create a Map instance using geemap
Map = geemap.Map(center=[35, 105], zoom=4)  # Centered on China

# Add nightlight mosaics as layers
Map.addLayer(old_nightlights, nightlight_vis_params, 'Old Nightlights')
Map.addLayer(new_nightlights, nightlight_vis_params, 'New Nightlights')

# Create separate layers for each state, colored based on radiance following the previously generated colormap
for item in final_list:
    name, radiance, color = item
    selected_feature = radiance_collection.filter(ee.Filter.eq('State Name', name))
    Map.addLayer(selected_feature, {'color': color}, name)

# Add layer control to map
Map.addLayerControl()

# Create legend variables from existing colormap
legend_keys = ['>10% decrease', '5-10% decrease', '<5% difference', '5-10% increase', '>10% increase']
legend_colors = radiance_colormap

# Add legend to map
Map.add_legend(
    keys = legend_keys, colors = legend_colors, position = 'bottomleft'
)

# View map
Map

Map(center=[35, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…